# Implementing an Animal Expert System

In this sample, we will implement a simple knowledge-based system to determine an animal based on some physical characteristics.

The system can be represented by the following AND-OR tree (this is a part of the whole tree, we can easily add some more rules).

![](images/AND-OR-Tree.png)